<a href="https://colab.research.google.com/github/asadyousuf-dare/Thesis-work/blob/main/Tuning_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)
from tensorflow.keras.layers import Embedding
from IPython.display import clear_output

from keras.layers import Dense
from tensorflow.keras import layers
%config InlineBackend.figure_format = 'retina'

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
plt.rcParams['figure.figsize']=[14,10]
import seaborn as sns

In [11]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 13.8 MB/s 


In [12]:
import keras_tuner as kt
from tensorflow import keras

In [2]:
#pip install keras-tuner --upgrade

In [16]:
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [6]:
!#pip install lrcurve

In [13]:
df = pd.read_csv('/content/training.csv')

In [24]:
df_cv= pd.read_csv('/content/testing.csv')

In [25]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 3)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [26]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    directory='ANN',
    project_name='ANN_learning')

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [20]:
#Traing data seperation

In [28]:
# Separate Target Variable and Predictor Variables
TargetVariable=['MR']
Predictors=['fc20', 'fy20', 'cover', 'depth', 'area' ]

X_train=df[Predictors].values
y_train=df[TargetVariable].values
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()
 
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_train)
TargetVarScalerFit=TargetVarScaler.fit(y_train)
 
# Generating the standardized values of X and y
X_train=PredictorScalerFit.transform(X_train)
y_train=TargetVarScalerFit.transform(y_train)
 


In [22]:
#Testing data seperation

In [29]:
TargetVariable=['MR']
Predictors=['fc20', 'fy20', 'cover', 'depth', 'area' ]

X_test=df_cv[Predictors].values
y_test=df_cv[TargetVariable].values
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()
 
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_test)
TargetVarScalerFit=TargetVarScaler.fit(y_test)
 
# Generating the standardized values of X and y
X_test=PredictorScalerFit.transform(X_test)
y_test=TargetVarScalerFit.transform(y_test)

In [ ]:
#Traing data seperation

In [30]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
best_model = tuner.get_best_models()[0]

Trial 5 Complete [00h 00m 01s]
val_loss: 0.04478335380554199

Best val_loss So Far: 0.04478335380554199
Total elapsed time: 00h 00m 15s
INFO:tensorflow:Oracle triggered exit


In [31]:
tuner.search_space_summary() 

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [32]:
tuner.results_summary()

Results summary
Results in ANN/ANN_learning
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 3
units_0: 384
units_1: 64
learning_rate: 0.01
units_2: 352
Score: 0.04478335380554199
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
units_1: 64
learning_rate: 0.001
Score: 0.04607729986310005
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
units_1: 384
learning_rate: 0.001
Score: 0.05256080999970436
Trial summary
Hyperparameters:
num_layers: 3
units_0: 64
units_1: 352
learning_rate: 0.001
units_2: 32
Score: 0.05330803245306015
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
units_1: 416
learning_rate: 0.001
units_2: 160
Score: 0.05649758502840996
